# Script for loading BingCovid source data from GitHub into SQL Server

In [1]:
import pyodbc
import pandas as pd

Read the raw CSV data source file from GitHub and filter data just for United States & States

In [2]:
BingCovidURL='https://raw.githubusercontent.com/microsoft/Bing-COVID-19-Data/master/data/Bing-COVID19-Data.csv'

In [3]:
BingCovidData = pd.read_csv(BingCovidURL, iterator=True, chunksize=1000)
df = pd.concat([chunk[(chunk['Country_Region'] =="United States") & (chunk['AdminRegion2'].notnull()==False)] for chunk in BingCovidData])
df.head(10)

,ID,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,Latitude,Longitude,ISO2,ISO3,Country_Region,AdminRegion1,AdminRegion2
252561,339285,01/23/2020,1,NaN,0.0,NaN,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252562,339286,01/24/2020,1,0.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252563,339287,01/25/2020,2,1.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252564,339288,01/26/2020,2,0.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252565,339289,01/27/2020,5,3.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252566,339290,01/28/2020,5,0.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252567,339291,01/29/2020,5,0.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252568,339292,01/30/2020,5,0.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252569,339293,01/31/2020,6,1.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN
252570,339294,02/01/2020,7,1.0,0.0,0.0,NaN,NaN,39.49591,-98.98998,US,USA,United States,NaN,NaN


Remove not needed columns

In [4]:
del df['ID']
del df['Latitude']
del df['Longitude']
del df['AdminRegion2']
df.head(10)

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,ISO2,ISO3,Country_Region,AdminRegion1
252561,01/23/2020,1,NaN,0.0,NaN,NaN,NaN,US,USA,United States,NaN
252562,01/24/2020,1,0.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252563,01/25/2020,2,1.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252564,01/26/2020,2,0.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252565,01/27/2020,5,3.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252566,01/28/2020,5,0.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252567,01/29/2020,5,0.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252568,01/30/2020,5,0.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252569,01/31/2020,6,1.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN
252570,02/01/2020,7,1.0,0.0,0.0,NaN,NaN,US,USA,United States,NaN


Check latest date for compete data for all states

In [5]:
maxDate=max(df['Updated'])
print(maxDate)

rowCount=df[(df['Updated']==maxDate) & (df['AdminRegion1'].isnull()==False)].count()['AdminRegion1']
print(rowCount)

if rowCount<=51:
    df.drop(df.loc[df['Updated']==maxDate].index, inplace=True)

09/13/2020
16


Handling Null values 

In [6]:
dfClean=df.where(pd.notnull(df),None)
dfClean.head(10)

,Updated,Confirmed,ConfirmedChange,Deaths,DeathsChange,Recovered,RecoveredChange,ISO2,ISO3,Country_Region,AdminRegion1
252561,01/23/2020,1,None,0,None,None,None,US,USA,United States,None
252562,01/24/2020,1,0,0,0,None,None,US,USA,United States,None
252563,01/25/2020,2,1,0,0,None,None,US,USA,United States,None
252564,01/26/2020,2,0,0,0,None,None,US,USA,United States,None
252565,01/27/2020,5,3,0,0,None,None,US,USA,United States,None
252566,01/28/2020,5,0,0,0,None,None,US,USA,United States,None
252567,01/29/2020,5,0,0,0,None,None,US,USA,United States,None
252568,01/30/2020,5,0,0,0,None,None,US,USA,United States,None
252569,01/31/2020,6,1,0,0,None,None,US,USA,United States,None
252570,02/01/2020,7,1,0,0,None,None,US,USA,United States,None


In [7]:
BingCovidList=dfClean.values.tolist()

Connect to local SQL Server instance, dump & fill table

In [8]:
conn = pyodbc.connect('Driver={SQL Server};'
                    'Server=DESKTOP-BME09T7\MSSQLSERVER01;'
                    'Database=Covid19_NursingHomes;'
                    'Trusted_Connection=yes;', autocommit=True)
cursor = conn.cursor()

In [9]:
cursor.execute("TRUNCATE TABLE BingCovid")

In [10]:
cursor.executemany("INSERT INTO BingCovid(Updated,"
                       "Confirmed,ConfirmedChange,Deaths,DeathsChange,"
                       "Recovered,RecoveredChange,ISO2,ISO3,"
                       "Country_Region,State) VALUES(?,?,?,?,?,?,?,?,?,?,?)",
                       BingCovidList)

In [11]:
cursor.rowcount

-1